In [1]:
import mysql.connector as sql

# MySQL Credentials
mydb = sql.connect(
    host="localhost",
    user="root",
    password="11111111"
)

# Creating a cursor object to interact with the database
mycursor = mydb.cursor()

# Execute CREATE DATABASE query
mycursor.execute("CREATE DATABASE IF NOT EXISTS Registration_Form")

# Select the created database
mycursor.execute("USE Registration_Form")

# Execute CREATE TABLE query
create_table_query = """
CREATE TABLE IF NOT EXISTS Users_Data (
    id INT AUTO_INCREMENT PRIMARY KEY,
    first_name VARCHAR(255),
    last_name VARCHAR(255),
    dob DATE,
    gender VARCHAR(10),
    email_address VARCHAR(255),
    phone_number VARCHAR(20),
    residential_address VARCHAR(255),
    user_name VARCHAR(50),
    password VARCHAR(50)
)
"""

mycursor.execute(create_table_query)

# Committing the changes
mydb.commit()

# Close the cursor and database connection
mycursor.close()
mydb.close()

In [2]:
import tkinter as tk
from tkinter import ttk, messagebox
import mysql.connector as sql

# Function to insert data into the database
def insert_data():
    # Retrieve data from entry widgets
    first_name = entry_first_name.get()
    last_name = entry_last_name.get()
    dob = entry_dob.get()
    gender = entry_gender.get()
    email_address = entry_email.get()
    phone_number = entry_phone.get()
    residential_address = entry_address.get()
    user_name = entry_username.get()
    password = entry_password.get()

    # MySQL Credentials
    mydb = sql.connect(
        host="localhost",
        user="root",
        password="11111111",
        database="Registration_Form"  # Specify the database name
    )

    # Creating a cursor object to interact with the database
    mycursor = mydb.cursor()

    # Execute INSERT query
    insert_query = """
    INSERT INTO Users_Data (first_name, last_name, dob, gender, email_address, phone_number, residential_address, user_name, password)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    data = (first_name, last_name, dob, gender, email_address, phone_number, residential_address, user_name, password)

    try:
        mycursor.execute(insert_query, data)
        mydb.commit()
        messagebox.showinfo("Success", "Data inserted successfully!")

        # Update the Treeview with the inserted data
        update_treeview()

    except sql.Error as e:
        mydb.rollback()
        messagebox.showerror("Error", f"Error inserting data: {e}")
    finally:
        # Close the cursor and database connection
        mycursor.close()
        mydb.close()

# Function to update the Treeview with data from the database
def update_treeview():
    # Clear existing data in the Treeview
    for row in tree.get_children():
        tree.delete(row)

    # MySQL Credentials
    mydb = sql.connect(
        host="localhost",
        user="root",
        password="11111111",
        database="Registration_Form"  # Specify the database name
    )

    # Creating a cursor object to interact with the database
    mycursor = mydb.cursor()

    try:
        # Execute SELECT query
        select_query = "SELECT * FROM Users_Data"
        mycursor.execute(select_query)

        # Fetch all rows
        rows = mycursor.fetchall()

        # Reverse the order of rows to show recent data at the top
        rows = reversed(rows)

        # Get the next available ID
        next_id_query = "SELECT MAX(ID) FROM Users_Data"
        mycursor.execute(next_id_query)
        max_id_result = mycursor.fetchone()[0]

        # Set next_id to 1 if the table is empty
        next_id = max_id_result + 1 if max_id_result is not None else 1

        # Insert data into the Treeview
        for row in rows:
            tree.insert("", "end", values=row)

        # Update the entry_id widget with the next available ID
        entry_id.config(state="normal")
        entry_id.delete(0, tk.END)
        entry_id.insert(0, next_id)
        entry_id.config(state="readonly")

        # Bind the function to the Treeview selection event
        tree.bind("<ButtonRelease-1>", select_record)

    except sql.Error as e:
        # Handle the error (you might want to show an error message)
        messagebox.showerror("Error", f"Error fetching data: {e}")

    finally:
        # Close the cursor and database connection
        mycursor.close()
        mydb.close()

# Function to delete selected item from the Treeview
def delete_data():
    selected_item = tree.selection()
    if not selected_item:
        messagebox.showinfo("Info", "Please select a record to delete.")
        return

    # MySQL Credentials
    mydb = sql.connect(
        host="localhost",
        user="root",
        password="11111111",
        database="Registration_Form"  # Specify the database name
    )

    # Creating a cursor object to interact with the database
    mycursor = mydb.cursor()

    try:
        # Extract the ID of the selected item
        selected_id = tree.item(selected_item, 'values')[0]

        # Execute DELETE query
        delete_query = "DELETE FROM Users_Data WHERE ID = %s"
        mycursor.execute(delete_query, (selected_id,))
        mydb.commit()

        messagebox.showinfo("Success", "Data deleted successfully!")

        # Update the Treeview after deletion
        update_treeview()

    except sql.Error as e:
        mydb.rollback()
        messagebox.showerror("Error", f"Error deleting data: {e}")
    finally:
        # Close the cursor and database connection
        mycursor.close()
        mydb.close()

# Function to populate entry fields when a record is selected
def select_record(event):
    selected_item = tree.selection()
    if not selected_item:
        return

    # Retrieve the values of the selected item
    selected_values = tree.item(selected_item, 'values')

    # Populate entry widgets with selected values for editing
    entry_id.config(state="normal")
    entry_id.delete(0, tk.END)
    entry_id.insert(0, selected_values[0])
    entry_id.config(state="readonly")

    entry_first_name.delete(0, tk.END)
    entry_first_name.insert(0, selected_values[1])

    entry_last_name.delete(0, tk.END)
    entry_last_name.insert(0, selected_values[2])

    entry_dob.delete(0, tk.END)
    entry_dob.insert(0, selected_values[3])

    entry_gender.delete(0, tk.END)
    entry_gender.insert(0, selected_values[4])

    entry_email.delete(0, tk.END)
    entry_email.insert(0, selected_values[5])

    entry_phone.delete(0, tk.END)
    entry_phone.insert(0, selected_values[6])

    entry_address.delete(0, tk.END)
    entry_address.insert(0, selected_values[7])

    entry_username.delete(0, tk.END)
    entry_username.insert(0, selected_values[8])

    entry_password.delete(0, tk.END)
    entry_password.insert(0, selected_values[9])

# Function to update the selected item in the Treeview
def update_data():
    # Retrieve data from entry widgets
    selected_id = entry_id.get()
    first_name = entry_first_name.get()
    last_name = entry_last_name.get()
    dob = entry_dob.get()
    gender = entry_gender.get()
    email_address = entry_email.get()
    phone_number = entry_phone.get()
    residential_address = entry_address.get()
    user_name = entry_username.get()
    password = entry_password.get()

    # MySQL Credentials
    mydb = sql.connect(
        host="localhost",
        user="root",
        password="11111111",
        database="Registration_Form"  # Specify the database name
    )

    # Creating a cursor object to interact with the database
    mycursor = mydb.cursor()

    # Execute UPDATE query
    update_query = """
    UPDATE Users_Data
    SET first_name=%s, last_name=%s, dob=%s, gender=%s, email_address=%s,
        phone_number=%s, residential_address=%s, user_name=%s, password=%s
    WHERE ID=%s
    """
    data = (first_name, last_name, dob, gender, email_address, phone_number, residential_address, user_name, password, selected_id)

    try:
        mycursor.execute(update_query, data)
        mydb.commit()
        messagebox.showinfo("Success", "Data updated successfully!")

        # Update the Treeview after update
        update_treeview()

    except sql.Error as e:
        mydb.rollback()
        messagebox.showerror("Error", f"Error updating data: {e}")
    finally:
        # Close the cursor and database connection
        mycursor.close()
        mydb.close()

# Create the main window
app = tk.Tk()
app.title("Registration Form")

# Create entry widgets and labels for each field
label_id = tk.Label(app, text="ID:")
entry_id = tk.Entry(app, state="readonly")  # ID entry is read-only

label_first_name = tk.Label(app, text="First Name:")
entry_first_name = tk.Entry(app)

label_last_name = tk.Label(app, text="Last Name:")
entry_last_name = tk.Entry(app)

label_dob = tk.Label(app, text="Date of Birth:")
entry_dob = tk.Entry(app)

label_gender = tk.Label(app, text="Gender:")
entry_gender = tk.Entry(app)

label_email = tk.Label(app, text="Email Address:")
entry_email = tk.Entry(app)

label_phone = tk.Label(app, text="Phone Number:")
entry_phone = tk.Entry(app)

label_address = tk.Label(app, text="Residential Address:")
entry_address = tk.Entry(app)

label_username = tk.Label(app, text="User Name:")
entry_username = tk.Entry(app)

label_password = tk.Label(app, text="Password:")
entry_password = tk.Entry(app, show="*")

# Create buttons to submit, update, and delete the data
create_button = tk.Button(app, text="Create", command=insert_data, width=10)
update_button = tk.Button(app, text="Update", command=update_data, width=10)
delete_button = tk.Button(app, text="Delete", command=delete_data, width=10)

# Create Treeview widget
columns = ("ID", "First Name", "Last Name", "DOB", "Gender", "Email", "Phone", "Address", "Username", "Password")
tree = ttk.Treeview(app, columns=columns, show="headings")

# Add column headings
for col in columns:
    tree.heading(col, text=col)
    tree.column(col, width=80)  # Adjust the column width as needed

# Update the Treeview with the data when the application starts
update_treeview()    
    
# Arrange widgets using grid layout
rows = 11  # Increase the number of rows to accommodate the new ID field
columns = 3  # Increase the number of columns for the new update button

# Center all elements using grid
for row in range(rows):
    app.grid_rowconfigure(row, weight=1)
for col in range(columns):
    app.grid_columnconfigure(col, weight=1)

label_id.grid(row=0, column=0, padx=10, pady=5, sticky="e")
entry_id.grid(row=0, column=1, padx=10, pady=5, sticky="w")

label_first_name.grid(row=1, column=0, padx=10, pady=5, sticky="e")
entry_first_name.grid(row=1, column=1, padx=10, pady=5, sticky="w")

label_last_name.grid(row=2, column=0, padx=10, pady=5, sticky="e")
entry_last_name.grid(row=2, column=1, padx=10, pady=5, sticky="w")

label_dob.grid(row=3, column=0, padx=10, pady=5, sticky="e")
entry_dob.grid(row=3, column=1, padx=10, pady=5, sticky="w")

label_gender.grid(row=4, column=0, padx=10, pady=5, sticky="e")
entry_gender.grid(row=4, column=1, padx=10, pady=5, sticky="w")

label_email.grid(row=5, column=0, padx=10, pady=5, sticky="e")
entry_email.grid(row=5, column=1, padx=10, pady=5, sticky="w")

label_phone.grid(row=6, column=0, padx=10, pady=5, sticky="e")
entry_phone.grid(row=6, column=1, padx=10, pady=5, sticky="w")

label_address.grid(row=7, column=0, padx=10, pady=5, sticky="e")
entry_address.grid(row=7, column=1, padx=10, pady=5, sticky="w")

label_username.grid(row=8, column=0, padx=10, pady=5, sticky="e")
entry_username.grid(row=8, column=1, padx=10, pady=5, sticky="w")

label_password.grid(row=9, column=0, padx=10, pady=5, sticky="e")
entry_password.grid(row=9, column=1, padx=10, pady=5, sticky="w")

# Center the Create, Update, and Delete buttons in the new columns with columnspan
create_button.grid(row=10, column=0, columnspan=1, pady=10, sticky="nsew")
update_button.grid(row=10, column=1, columnspan=1, pady=10, sticky="nsew")  # Add the update button
delete_button.grid(row=10, column=2, columnspan=1, pady=10, sticky="nsew")  # Add the delete button

# Place the Treeview widget below the buttons
tree.grid(row=11, column=0, columnspan=3, pady=10, sticky="nsew")

# Start the main loop
app.mainloop()